In [1]:
import mediapipe as mp
import cv2
from matplotlib import pyplot as plt

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 이미지를 불러오기
image = cv2.imread('models_eva.png')
# BGR 이미지를 RGB로 변환
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 포즈 랜드마크 감지
results = pose.process(image)

# 랜드마크를 이미지에 그리기
mp_drawing = mp.solutions.drawing_utils
mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

# 결과 보기
plt.imshow(image)
plt.show()


ModuleNotFoundError: No module named 'mediapipe'

In [6]:
import cv2 as cv
import numpy as np
import mediapipe as mp
from PIL import Image

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def apply_clothes_on_person(person_img_path, clothes_img_path):
    # 1. 사용자 이미지 로드 및 포즈 추출
    person_img = cv.imread(person_img_path)
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
    results = pose.process(cv.cvtColor(person_img, cv.COLOR_BGR2RGB))
    
    if not results.pose_landmarks:
        raise Exception("No pose landmarks found in the image.")
    
    # 2. 옷 이미지 처리
    clothes_img = Image.open(clothes_img_path).convert("RGBA")
    
    # 3. 사용자의 어깨 및 힙 좌표 얻기
    shoulder_right = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    shoulder_left = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    
    # 4. 옷 이미지 조정 및 사용자 이미지에 입히기
    person_img_width, person_img_height = person_img.shape[1], person_img.shape[0]
    
    shoulder_right_x, shoulder_right_y = int(shoulder_right.x * person_img_width), int(shoulder_right.y * person_img_height)
    shoulder_left_x, shoulder_left_y = int(shoulder_left.x * person_img_width), int(shoulder_left.y * person_img_height)
    
    shoulder_width = np.linalg.norm([shoulder_left_x - shoulder_right_x, shoulder_left_y - shoulder_right_y])
    clothes_width = shoulder_width * 2  # 옷의 너비를 어깨 너비의 2배로 설정
    scale_factor = clothes_width / clothes_img.width
    resized_clothes_img = clothes_img.resize((int(clothes_width), int(clothes_img.height * scale_factor)), Image.Resampling.LANCZOS)
    
    # 어깨 중앙점 계산
    shoulders_center_x = (shoulder_left_x + shoulder_right_x) / 2
    
    # 옷 이미지의 시작 x 좌표를 어깨 중앙점을 기준으로 조정
    overlay_start_x = int(shoulders_center_x - clothes_width / 2)
    overlay_start_y = int(shoulder_left_y)
    
    person_img_pil = Image.fromarray(cv.cvtColor(person_img, cv.COLOR_BGR2RGB))
    person_img_pil.paste(resized_clothes_img, (overlay_start_x, overlay_start_y), resized_clothes_img)
    person_img_final = cv.cvtColor(np.array(person_img_pil), cv.COLOR_RGB2BGR)
    
    # 5. 결과 이미지 출력 또는 저장
    cv.imwrite('result_with_clothes.png', person_img_final)
    cv.imshow("Result", person_img_final)
    cv.waitKey(0)
    cv.destroyAllWindows()


# 함수 사용 예시
apply_clothes_on_person('models_eva.png', 'clothes/pink_shirt.png')


In [8]:
def apply_clothes_on_person(person_img_path, clothes_img_path, shift_up_factor=0.1):
    # 1. 사용자 이미지 로드 및 포즈 추출
    person_img = cv.imread(person_img_path)
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
    results = pose.process(cv.cvtColor(person_img, cv.COLOR_BGR2RGB))
    
    if not results.pose_landmarks:
        raise Exception("No pose landmarks found in the image.")
    
    # 2. 옷 이미지 처리
    clothes_img = Image.open(clothes_img_path).convert("RGBA")
    
    # 3. 사용자의 어깨 및 힙 좌표 얻기
    shoulder_right = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    shoulder_left = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    
    # 4. 옷 이미지 조정 및 사용자 이미지에 입히기
    person_img_width, person_img_height = person_img.shape[1], person_img.shape[0]
    
    shoulder_right_x, shoulder_right_y = int(shoulder_right.x * person_img_width), int(shoulder_right.y * person_img_height)
    shoulder_left_x, shoulder_left_y = int(shoulder_left.x * person_img_width), int(shoulder_left.y * person_img_height)
    
    shoulder_width = np.linalg.norm([shoulder_left_x - shoulder_right_x, shoulder_left_y - shoulder_right_y])
    clothes_width = shoulder_width * 2  # 옷의 너비를 어깨 너비의 2배로 설정
    scale_factor = clothes_width / clothes_img.width
    resized_clothes_img = clothes_img.resize((int(clothes_width), int(clothes_img.height * scale_factor)), Image.Resampling.LANCZOS)
    
    # 어깨 중앙점 계산
    shoulders_center_x = (shoulder_left_x + shoulder_right_x) / 2
    
    # 옷 이미지의 시작 x 좌표를 어깨 중앙점을 기준으로 조정
    overlay_start_x = int(shoulders_center_x - clothes_width / 2)
    
    # 옷을 더 위로 올리기 위해 overlay_start_y 값을 조정
    overlay_start_y_adjusted = int(shoulder_left_y - (clothes_img.height * scale_factor * shift_up_factor))
    
    person_img_pil = Image.fromarray(cv.cvtColor(person_img, cv.COLOR_BGR2RGB))
    person_img_pil.paste(resized_clothes_img, (overlay_start_x, overlay_start_y_adjusted), resized_clothes_img)
    person_img_final = cv.cvtColor(np.array(person_img_pil), cv.COLOR_RGB2BGR)
    
    # 5. 결과 이미지 출력 또는 저장
    cv.imwrite('result_with_clothes.png', person_img_final)
    cv.imshow("Result", person_img_final)
    cv.waitKey(0)
    cv.destroyAllWindows()

# 함수 사용 예시: 옷을 10% 더 위로 올림
apply_clothes_on_person('models_eva.png', 'clothes/pink_shirt.png', 0.1)


In [13]:
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image

# MediaPipe 포즈 객체 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

def overlay_on_image(background_img, overlay_img, point1, point2, overlay_alpha=None):
    # 랜드마크 포인트 사이의 거리를 계산하여 옷 이미지 크기 조정
    p1, p2 = point1, point2
    distance = np.linalg.norm(np.array(p1) - np.array(p2))
    scale_factor = distance / overlay_img.shape[1]
    new_size = (int(overlay_img.shape[1] * scale_factor), int(overlay_img.shape[0] * scale_factor))
    resized_overlay = cv2.resize(overlay_img, new_size, interpolation=cv2.INTER_AREA)
    
    # 옷 이미지의 좌표를 계산(옷 이미지를 중앙에 위치시키기 위함)
    mid_point = ((p1[0] + p2[0]) // 2, (p1[1] + p2[1]) // 2)
    top_left = (mid_point[0] - resized_overlay.shape[1] // 2, mid_point[1] - resized_overlay.shape[0] // 2)
    
    # 옷 이미지 합성
    if overlay_alpha is not None:
        # Alpha 마스크를 사용한 옷 이미지 합성
        for c in range(0, 3):
            background_img[top_left[1]:top_left[1]+new_size[1], top_left[0]:top_left[0]+new_size[0], c] = \
                resized_overlay[:, :, c] * (overlay_alpha / 255.0) + \
                background_img[top_left[1]:top_left[1]+new_size[1], top_left[0]:top_left[0]+new_size[0], c] * (1.0 - overlay_alpha / 255.0)
    else:
        # 옷 이미지가 알파 마스크를 포함하지 않는 경우 단순 합성
        background_img[top_left[1]:top_left[1]+new_size[1], top_left[0]:top_left[0]+new_size[0]] = resized_overlay
        
    return background_img

# 사진 불러오기 또는 카메라로 캡쳐하기
# 이 부분은 사용자 입력에 따라 이미지를 불러오는 로직입니다.
# 예를 들어, 파일로부터 이미지를 불러오는 경우:
input_img_path = "models_eva.png"  # 사용자로부터 입력받거나 파일 다이얼로그를 통해 얻음
input_img = cv2.imread(input_img_path)

# 포즈 랜드마크를 찾아낸 뒤, 옷 이미지 합성
# 이미지에서 포즈 랜드마크를 추출하는 과정이 필요합니다. 아래는 랜드마크 추출 예시입니다.
# (실제 사용시에는 MediaPipe의 포즈 추출 결과를 사용해야 합니다.)
landmarks = pose.process(cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)).pose_landmarks
if landmarks:
    shoulder_right = [landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
    shoulder_left = [landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y]

# 옷 이미지 및 알파 마스크 불러오기
# 예를 들어, PNG 형식의 상의 이미지:
overlay_img_path = "clothes/pink_shirt.png"

# (계속) 옷 이미지 및 알파 마스크 불러오기
overlay_img = cv2.imread(overlay_img_path, -1)  # -1 플래그로 알파 채널 포함하여 이미지 로드
overlay_alpha = overlay_img[:, :, 3]  # 알파 채널 분리
overlay_img = overlay_img[:, :, :3]  # RGB 채널만 남김

# 상의 이미지 입히기를 위한 랜드마크 포인트 지정
# 예를 들어, 양쪽 어깨 랜드마크
shoulder_right = [int(landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * input_img.shape[1]), int(landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * input_img.shape[0])]
shoulder_left = [int(landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * input_img.shape[1]), int(landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * input_img.shape[0])]

# 상의 이미지 합성 함수 호출
result_img = overlay_on_image(input_img, overlay_img, shoulder_left, shoulder_right, overlay_alpha)

# 결과 이미지 표시 및 저장
cv2.imshow('Result', result_img)
cv2.waitKey(0)  # 사용자가 키를 누를 때까지 대기
cv2.destroyAllWindows()  # 모든 창 닫기

result_img_path = "result_image.jpg"  # 결과 이미지 저장 경로
cv2.imwrite(result_img_path, result_img)  # 결과 이미지 파일로 저장

ValueError: operands could not be broadcast together with shapes (262,202) (497,384) 